In [1]:
from crawl import Crawler
from parse import find_all_safely
import pandas as pd
import os
from setup_db import DB, setup_db, cursor_as_df, execute_to_df

In [2]:
db = setup_db()

In [3]:
# db.close()

In [4]:
db.select_all("jobs")

,job_id,user_created,active,time_created,last_run,job_name
0,1,1,1,2019-10-10 00:00:00,None,test_job


In [5]:
db.select_all("requests")

,request_id,job_id,start_city,end_city,date
0,1,1,1915,1815,2020-01-02 00:00:00
1,2,1,1915,7588,2020-01-02 00:00:00
2,3,1,1915,7588,2020-01-03 00:00:00


SELECT 
strftime('%d.%m.%Y',t2.time_created) as rideDate,
t2.start_city as departureCity,
t2.end_city as arrivalCity
FROM jobs as t1
left join requests t2 ON t1.job_id = t2.job_id
WHERE t1.active = 1


In [6]:
params_from_db = db.c.execute('''SELECT 
strftime("%d.%m.%Y",t2.date) as rideDate,
t2.start_city as departureCity,
t2.end_city as arrivalCity
FROM jobs as t1
left join requests as t2 ON t1.job_id = t2.job_id
WHERE t1.active = 1''')
colnames = [i[0] for i in params_from_db.description]
values = params_from_db.fetchall()

In [7]:
print(colnames)
print(values)

['rideDate', 'departureCity', 'arrivalCity']
[('02.01.2020', 1915, 1815), ('02.01.2020', 1915, 7588), ('03.01.2020', 1915, 7588)]


In [8]:
constant_search_params = {
    "adult": "1",
    "_locale": "pl"
}


In [9]:
params_list = []
for single_params in values:
    params_list.append({"rideDate": single_params[0],
                       "departureCity": single_params[1],
                       "arrivalCity": single_params[2]})
    
params_list = [{**i, **constant_search_params} for i in params_list]

In [10]:
params_list

[{'rideDate': '02.01.2020',
  'departureCity': 1915,
  'arrivalCity': 1815,
  'adult': '1',
  '_locale': 'pl'},
 {'rideDate': '02.01.2020',
  'departureCity': 1915,
  'arrivalCity': 7588,
  'adult': '1',
  '_locale': 'pl'},
 {'rideDate': '03.01.2020',
  'departureCity': 1915,
  'arrivalCity': 7588,
  'adult': '1',
  '_locale': 'pl'}]

In [11]:
crawler = Crawler(params_list[1])
crawler.get_all()
crawler.get_pandas_df()

,departure_time,arrival_time,departure_station,arrival_station,price
0,00:30,04:00,Kraków MDA,Łódź Fabryczna,"69,99"
1,07:05,11:25,Kraków MDA,Łódź Fabryczna,"69,99"
2,12:00,16:00,Kraków MDA,Łódź Fabryczna,None
3,15:10,19:25,Kraków MDA,Łódź Fabryczna,"69,99"
4,19:15,23:50,Kraków MDA,Łódź Fabryczna,None
5,23:59,04:15,Kraków MDA,Łódź Fabryczna,"45,99"
6,00:30,04:00,Kraków MDA,Łódź Fabryczna,"69,99"


In [12]:
results = []
for search_params in params_list:
    crawler = Crawler(search_params)
    crawler.get_all()
    results.append(crawler.get_pandas_df())
    
results

[  departure_time arrival_time departure_station     arrival_station   price
 0          00:30        09:35        Kraków MDA              Gdańsk  129,99
 1          00:30        11:45        Kraków MDA  Gdańsk Hala Olivia  129,98,
   departure_time arrival_time departure_station arrival_station  price
 0          00:30        04:00        Kraków MDA  Łódź Fabryczna  69,99
 1          07:05        11:25        Kraków MDA  Łódź Fabryczna  69,99
 2          12:00        16:00        Kraków MDA  Łódź Fabryczna   None
 3          15:10        19:25        Kraków MDA  Łódź Fabryczna  69,99
 4          19:15        23:50        Kraków MDA  Łódź Fabryczna   None
 5          23:59        04:15        Kraków MDA  Łódź Fabryczna  45,99
 6          00:30        04:00        Kraków MDA  Łódź Fabryczna  69,99,
   departure_time arrival_time departure_station arrival_station  price
 0          00:30        04:00        Kraków MDA  Łódź Fabryczna  69,99
 1          07:05        11:25        Kraków MD